In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [62]:
df = pd.read_csv('googleplaystore.csv')
df.shape

(10841, 13)

In [63]:
df.isna().sum()

App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       1
Genres               0
Last Updated         0
Current Ver          8
Android Ver          3
dtype: int64

In [64]:
# data error

df = df.drop(index=10472, axis='rows')

**TASK 1**

In [65]:
"""
Read the apps.csv file and clean the Installs
column to convert it into integer data type. 
Save your answer as a DataFrame apps. 
Going forward, you will do all your analysis on the apps DataFrame.
"""

'\nRead the apps.csv file and clean the Installs\ncolumn to convert it into integer data type. \nSave your answer as a DataFrame apps. \nGoing forward, you will do all your analysis on the apps DataFrame.\n'

In [66]:
# cleaning the data format from unwanted symbol
# converting into integer data type (Installs and Price column)

df['Installs'] = df['Installs'].str.replace('+','').str.replace(',','').astype(int)
df['Price'] = df['Price'].str.replace('$','').str.replace('.','').astype(int)

**TASK 2**

In [67]:
"""
Find the number of apps in each category, the average price, and the average rating. 
Save your answer as a DataFrame app_category_info. 
Your should rename the four columns as: 

Category, Number of apps, Average price, Average rating.
"""

'\nFind the number of apps in each category, the average price, and the average rating. \nSave your answer as a DataFrame app_category_info. \nYour should rename the four columns as: \n\nCategory, Number of apps, Average price, Average rating.\n'

In [68]:
# using groupby method to find desired value

apps = df.groupby('Category', dropna=False).agg({'App':'count'})

app_category_info = df.groupby(by='Category', dropna=False).mean()
app_category_info = app_category_info.drop(columns='Installs')
app_category_info = app_category_info.rename(columns={'Rating':'Average Rating', 'Price':'Average Price'})
app_category_info = app_category_info[['Average Price','Average Rating']].round(2)
app_category_info = pd.concat([apps,app_category_info], axis='columns')
app_category_info = app_category_info.rename(columns={'App':'Number of Apps'})
app_category_info = app_category_info.reset_index()

app_category_info

,Category,Number of Apps,Average Price,Average Rating
0,ART_AND_DESIGN,65,9.18,4.36
1,AUTO_AND_VEHICLES,85,15.85,4.19
2,BEAUTY,53,0.00,4.28
3,BOOKS_AND_REFERENCE,231,51.85,4.35
4,BUSINESS,460,40.28,4.12
5,COMICS,60,0.00,4.16
6,COMMUNICATION,387,21.48,4.16
7,DATING,234,13.43,3.97
8,EDUCATION,156,11.51,4.39
9,ENTERTAINMENT,149,5.36,4.13


**TASK 3**

In [69]:
"""
Find the top 10 free FINANCE apps having the highest average sentiment score. 
Save your answer as a DataFrame top_10_user_feedback. 
Your answer should have exactly 10 rows and two columns named: 

App and Sentiment Score, where the average Sentiment Score is sorted from highest to lowest.
"""

'\nFind the top 10 free FINANCE apps having the highest average sentiment score. \nSave your answer as a DataFrame top_10_user_feedback. \nYour answer should have exactly 10 rows and two columns named: \n\nApp and Sentiment Score, where the average Sentiment Score is sorted from highest to lowest.\n'

In [70]:
# using the other dataset 
df1 = pd.read_csv('googleplaystore_user_reviews.csv')
df1.dropna(axis=0,inplace=True)

# changing App data type into str so that we can find the value
df1['App'] = df1['App'].astype(str)

# this is just an assumption for "finance" type because i can't find the similiar app with the other dataset (df)
# my assumptions are it contains the word "bank" or "finance" in it
fin = df1[(df1['App'].str.contains('Bank')) | (df1['App'].str.contains('Finance'))]

# replacing the sentiment value with numbers
fin['Sentiment'] = fin['Sentiment'].map({'Positive':1, 'Neutral':0, 'Negative':-1}).astype(int)

# using groupby method for calculation and sorting
top_10_user_feedback = fin.groupby('App').agg({'Sentiment':np.sum,'Sentiment_Polarity':np.mean, 
                        'Sentiment_Subjectivity':np.mean}).\
                        sort_values('Sentiment_Polarity',ascending=False).\
                        drop(columns=['Sentiment','Sentiment_Subjectivity']).\
                        rename(columns={'Sentiment_Polarity':'Sentiment Score'}).\
                        reset_index()

top_10_user_feedback

,App,Sentiment Score
0,BankMobile Vibe App,0.353455
1,GMAT Question Bank,0.268900
2,Fortune City - A Finance App,0.266966
3,BBVA Compass Banking,0.205590
4,Bank of America Mobile Banking,0.180027
5,HDFC Bank MobileBanking,0.111449
6,GoBank,0.039101
7,Citizens Bank Mobile Banking,0.013179
8,Bualuang mBanking,0.004322
9,Ecobank Mobile Banking,-0.014860


In [71]:
# the other top_10_user_feedback groupby method, calculating the highest sentiment score by Rating obtained

finance = df[(df['Category'] == 'FINANCE') & (df['Type'] == 'Free')]
finance.drop_duplicates(inplace=True)

top_10_user_feedback = finance[['App', 'Rating']].sort_values(by='Rating', ascending=False).head(10)
top_10_user_feedback = top_10_user_feedback.rename(columns={'Rating':'Sentiment Score'}).\
                        set_index(keys='App').reset_index().sort_index(ascending=True)

top_10_user_feedback

,App,Sentiment Score
0,DN Calculators,5.0
1,BxPort - Bitcoin Bx (Thailand),5.0
2,Free coupons and vouchers,5.0
3,BI APP,5.0
4,CT Checkout,5.0
5,CF,5.0
6,BK Gold App,5.0
7,"FD Calculator (EMI, SIP, RD & Loan Eligilibility)",5.0
8,EY Forensics,4.9
9,EC Tax,4.9
